In [39]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
%run ./common_init.ipynb

In [41]:
%autoreload 2
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.feature_selection import VarianceThreshold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from category_encoders import HashingEncoder, OneHotEncoder, OrdinalEncoder

# Load custom code
import kdd98.data_handler as dh
import kdd98.utils_transformer as ut
from kdd98.transformers import *
from kdd98.config import Config

In [42]:
# Where to save the figures
IMAGES_PATH = pathlib.Path(figure_output/'preprocessing')

pathlib.Path(IMAGES_PATH).mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = pathlib.Path(IMAGES_PATH, fig_id + "." + fig_extension)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [43]:
data_provider = dh.KDD98DataProvider("cup98LRN.txt")

In [11]:
learning_preprocessed = data_provider.preprocessed_data

In [12]:
from kdd98.transformers import ZipToCoords
from category_encoders import BinaryEncoder, OneHotEncoder

## Feature engineering

### Encode zip codes as coordinates
Instead of encoding the zip codes one-hot, which would lead to a significant increase in dimensionality (there are 16488 zip codes), they are transformed to their centroid coordinates. This gives an intuitive measure of geopgraphical relation between examples.

The coordinates are first searched for in a database from the 2018 US census, if not found there, the HERE geolocator web service is queried.

For military zip codes, there are no coordinates available. These are set to lat=0, lon=0.

In [13]:
len(learning_preprocessed.ZIP.unique())

16488

In [14]:
zip_to_coords = ColumnTransformer([("zip_to_coords", ZipToCoords(),
                                    ["ZIP", "STATE"])])
coords = zip_to_coords.fit_transform(learning_preprocessed)
coords_names = zip_to_coords.get_feature_names()
coords = pd.DataFrame(data=coords, index=learning_preprocessed.index, columns=coords_names)

In [15]:
learning_preprocessed = learning_preprocessed.merge(coords, on=learning_preprocessed.index.name)

In [16]:
learning_preprocessed.drop("ZIP", axis=1, inplace=True)

### Converting dates

There are several date features. ODATEDW is the date the record was added, DOB the birth date. ADATE_* and RDATE_* are from the promotion history. ADATE_* is the date of a mailing, RDATE_* the date the donation for the corresponding mailing was received. While these dates are not of particular interest (very low variance), the time it took to respond might be.
Furthermore, there are the features MINRDATE, MAXRDATE, MAXADATE, FISTDATE, NEXTDATE and LASTDATE coming from the giving history file.

In [17]:
print(dh.DATE_FEATURES)

['ODATEDW', 'DOB', 'ADATE_2', 'ADATE_3', 'ADATE_4', 'ADATE_5', 'ADATE_6', 'ADATE_7', 'ADATE_8', 'ADATE_9', 'ADATE_10', 'ADATE_11', 'ADATE_12', 'ADATE_13', 'ADATE_14', 'ADATE_15', 'ADATE_16', 'ADATE_17', 'ADATE_18', 'ADATE_19', 'ADATE_20', 'ADATE_21', 'ADATE_22', 'ADATE_23', 'ADATE_24', 'RDATE_3', 'RDATE_4', 'RDATE_5', 'RDATE_6', 'RDATE_7', 'RDATE_8', 'RDATE_9', 'RDATE_10', 'RDATE_11', 'RDATE_12', 'RDATE_13', 'RDATE_14', 'RDATE_15', 'RDATE_16', 'RDATE_17', 'RDATE_18', 'RDATE_19', 'RDATE_20', 'RDATE_21', 'RDATE_22', 'RDATE_23', 'RDATE_24', 'LASTDATE', 'MINRDATE', 'MAXRDATE', 'FISTDATE', 'NEXTDATE', 'MAXADATE']


The following helper function updates feature name lists and removes features that are no longer present because they were removed during preprocessing.

In [18]:
ALL_FEATURES = learning_preprocessed.columns.values.tolist()
def filter_features(features):
        return [f for f in features if f in ALL_FEATURES]

In [19]:
learning_preprocessed[filter_features(dh.DATE_FEATURES)]

,ODATEDW,ADATE_5,ADATE_7,ADATE_8,ADATE_9,ADATE_10,ADATE_11,ADATE_12,ADATE_13,ADATE_14,...,RDATE_16,RDATE_17,RDATE_18,RDATE_19,RDATE_21,RDATE_22,RDATE_24,LASTDATE,MINRDATE,MAXRDATE
CONTROLN,,,,,,,,,,,,,,,,,,,,,
95515,8901,9604,9602,9601,9511,9510,9510,9508,9507,9506,...,9505,9503,nan,nan,nan,nan,9406,9512,9208,9402
148535,9401,9604,9602,9601,9511,9510,9510,9509,nan,nan,...,9504,nan,nan,nan,nan,nan,nan,9512,9310,9512
15078,9001,9604,9602,9601,9511,nan,9510,9508,9507,9506,...,9504,nan,9501,nan,nan,9409,9406,9512,9111,9207
172556,8701,9604,9602,9601,9511,nan,9510,9508,9507,9506,...,9505,9503,nan,nan,9411,nan,nan,9512,8711,9411
7112,8601,9604,9512,9601,9511,9510,9509,9508,9502,9506,...,nan,nan,nan,nan,nan,nan,nan,9601,9310,9601
47784,9401,9604,9602,9601,9511,9510,9510,9509,9507,9506,...,nan,nan,9506,nan,nan,nan,9407,9506,9407,9412
62117,8701,nan,9602,9601,9511,9510,9510,9508,9507,9506,...,9504,nan,nan,nan,nan,9410,nan,9504,8705,9410
109359,9401,9604,9602,9601,9511,9510,9510,9509,9507,9506,...,9504,nan,nan,nan,nan,nan,9407,9508,9507,9508
75768,8801,9604,9602,9601,9511,9510,9509,9508,9507,9506,...,nan,nan,nan,nan,9411,nan,nan,9507,8809,9312


#### Donation history
From ADATE_*, the date a letter was sent, and RDATE_*, the date a donation was received, we can calculate the time in months it took to respond with a donation.

In [20]:
don_history = ColumnTransformer(
    [("months_to_donation",
      MonthsToDonation(reference_date=pd.datetime(1998, 6, 1)),
      filter_features(dh.PROMO_HISTORY_DATES + dh.GIVING_HISTORY_DATES))])
donation_history = don_history.fit_transform(learning_preprocessed)
donation_history_names = [n[n.find('__')+2:]
                 for n in don_history.get_feature_names()]
donation_history = pd.DataFrame(data=donation_history, index=learning_preprocessed.index, columns=donation_history_names)

In [21]:
learning_preprocessed = learning_preprocessed.merge(donation_history, on=learning_preprocessed.index.name)

In [22]:
learning_preprocessed.drop(filter_features(dh.PROMO_HISTORY_DATES + dh.GIVING_HISTORY_DATES), axis=1,inplace=True)

#### Time since donations, membership years
The time deltas for LASTDATE (last time a donation received), MINRDATE (when the smallest donation was received), MAXRDATE (when the largest donation was received) and MAXADATE (when the most recent promotion was sent) are expressed in months before the reference date (which is the sending date of the last promotion

Membership years are also computed against the reference date of the last promotion sent out.

In [23]:
t_deltas = ColumnTransformer(
    [("time_last_donation",
      DeltaTime(reference_date=pd.datetime(1997, 6, 1), unit="months"),
      filter_features(["LASTDATE", "MINRDATE", "MAXRDATE", "MAXADATE"])),
     ("membership_years",
      DeltaTime(reference_date=pd.datetime(1997, 6, 1), unit="years"),
      filter_features(["ODATEDW", "DOB"]))])
timedeltas = t_deltas.fit_transform(learning_preprocessed)
timedeltas_names = [n[n.find('__')+2:]
                 for n in t_deltas.get_feature_names()]
timedeltas = pd.DataFrame(data=timedeltas, index=learning_preprocessed.index, columns=timedeltas_names)

In [24]:
timedeltas

,LASTDATE_DELTA_MONTHS,MINRDATE_DELTA_MONTHS,MAXRDATE_DELTA_MONTHS,ODATEDW_DELTA_YEARS
CONTROLN,,,,
95515,18,58,40,9
148535,18,44,18,4
15078,18,67,59,8
172556,18,115,31,11
7112,17,44,17,12
47784,24,35,30,4
62117,26,121,32,11
109359,22,23,22,4
75768,23,105,42,10


In [25]:
learning_preprocessed = learning_preprocessed.merge(timedeltas, on=learning_preprocessed.index.name)

In [26]:
learning_preprocessed.drop(filter_features(["LASTDATE", "MINRDATE", "MAXRDATE", "MAXADATE", "ODATEDW"]), axis=1, inplace=True)

There are redundant features which can be safely removed, as is shown below:

1. FISTDATE and NEXTDATE are contained in TIMELAG, the number of months between first and second donation
2. DOB, the date of birth, is contained in the feature AGE

### Categoricals

In [27]:
CATEGORICAL_FEATURES = learning_preprocessed.select_dtypes(include="category").columns.values.tolist()
BE_CATEGORICALS = ['OSOURCE', 'TCODE', 'STATE', 'CLUSTER']
OHE_CATEGORICALS = [f for f in CATEGORICAL_FEATURES if f not in BE_CATEGORICALS]

#### Binary Encoding

Now, the nominals (categorical features with string levels) are worked on. Those categoricals with high cardinality (many levels) are bianry-encoded so as to not increase dimensionality too much.

The remaining features are one-hot encoded.
https://towardsdatascience.com/smarter-ways-to-encode-categorical-data-for-machine-learning-part-1-of-3-6dca2f71b159

In [28]:
binary_encode = ColumnTransformer([
                    ("be_osource", BinaryEncoder(handle_missing="return_nan"), filter_features(['OSOURCE'])),
                    ("be_state", BinaryEncoder(handle_missing="return_nan"), filter_features(['STATE'])),
                    ("be_cluster", BinaryEncoder(handle_missing="return_nan"), filter_features(['CLUSTER'])),
                    ("be_tcode", BinaryEncoder(handle_missing="return_nan"), filter_features(['TCODE']))
                ])
binary_encoded_categories = binary_encode.fit_transform(learning_preprocessed)
binary_encode_names = [n[n.find('__')+2:]
                 for n in binary_encode.get_feature_names()]
binary_encoded_categories = pd.DataFrame(data=binary_encoded_categories, index=learning_preprocessed.index, columns = binary_encode_names)


In [29]:
learning_preprocessed = learning_preprocessed.merge(binary_encoded_categories, on=learning_preprocessed.index.name)

In [30]:
learning_preprocessed.drop(filter_features(BE_CATEGORICALS), axis=1, inplace=True)

#### One-Hot Encoding

In [31]:
one_hot_encoding = ColumnTransformer([("oh",
                                       OneHotEncoder(
                                           use_cat_names=True,
                                           handle_missing="return_nan"),
                                       OHE_CATEGORICALS)])
oh_encoded_categories = one_hot_encoding.fit_transform(learning_preprocessed)
oh_encoded_categories_names = [n[n.find('__')+2:] for n in one_hot_encoding.get_feature_names()]
oh_encoded_categories = pd.DataFrame(data=oh_encoded_categories, index=learning_preprocessed.index, columns = oh_encoded_categories_names)

In [32]:
learning_preprocessed = learning_preprocessed.merge(oh_encoded_categories, on=learning_preprocessed.index.name)

In [33]:
learning_preprocessed.drop(OHE_CATEGORICALS, axis=1, inplace=True)

### Feature engineering combined

All the above steps are implemented in package kdd98. The data after feature engineering is readily available:

In [36]:
learning_numeric = data_provider.numeric_data

In [37]:
learning_numeric.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95412 entries, 95515 to 185114
Columns: 662 entries, RECINHSE to DOMAINUrbanicity_nan
dtypes: Int64(331), float64(38), int64(293)
memory usage: 512.7 MB
